# CSV vs Parquet vs Delta Lake

In [1]:
from deltalake import DeltaTable
import pandas as pd
from pathlib import Path

In [ ]:
path = f"{Path.home()}/data/deltalake_baseline_G1_1e9_1e2_0_0"

## CSV

In [4]:
%%time
(
    pd.read_csv(f"{Path.home()}/data/G1_1e9_1e2_0_0.csv")
    .query("id1 == 'id016'")
    .groupby("id2")
    .agg({"v1": "sum"})
)

KeyboardInterrupt: 

## CSV with usecols

## CSV => Parquet

In [6]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv

In [6]:
in_path = f"{Path.home()}/data/G1_1e8_1e2_0_0.csv"
out_path = f"{Path.home()}/data/G1_1e8_1e2_0_0.parquet"

In [7]:
# convert_options = pyarrow.csv.ConvertOptions()
# convert_options.column_types = {
#     'rate_code': pa.utf8(),
#     'store_and_fwd_flag': pa.utf8()
# }

writer = None
with pyarrow.csv.open_csv(in_path) as reader:
    for next_chunk in reader:
        if next_chunk is None:
            break
        if writer is None:
            writer = pq.ParquetWriter(out_path, next_chunk.schema)
        next_table = pa.Table.from_batches([next_chunk])
        writer.write_table(next_table)
writer.close()

In [3]:
%%time
(
    pd.read_parquet(f"{Path.home()}/data/G1_1e8_1e2_0_0.parquet", columns=["id1", "id2", "v1"])
    .query("id1 == 'id016'")
    .groupby("id2")
    .agg({"v1": "sum"})
)

CPU times: user 7.58 s, sys: 1.09 s, total: 8.67 s
Wall time: 4.68 s


,v1
id2,
id001,29918
id002,30343
id003,30180
id004,30581
id005,30769
...,...
id096,30011
id097,29728
id098,30131


In [4]:
in_path = f"{Path.home()}/data/G1_1e9_1e2_0_0.csv"
out_path = f"{Path.home()}/data/G1_1e9_1e2_0_0.parquet"

In [7]:
writer = None
with pyarrow.csv.open_csv(in_path) as reader:
    for next_chunk in reader:
        if next_chunk is None:
            break
        if writer is None:
            writer = pq.ParquetWriter(out_path, next_chunk.schema)
        next_table = pa.Table.from_batches([next_chunk])
        writer.write_table(next_table)
writer.close()

In [8]:
%%time
(
    pd.read_parquet(f"{Path.home()}/data/G1_1e9_1e2_0_0.parquet", columns=["id1", "id2", "v1"])
    .query("id1 == 'id016'")
    .groupby("id2")
    .agg({"v1": "sum"})
)

CPU times: user 1min 15s, sys: 31.5 s, total: 1min 47s
Wall time: 1min 27s


,v1
id2,
id001,301302
id002,299602
id003,300751
id004,300182
id005,298422
...,...
id096,299284
id097,300429
id098,301122
